In [283]:
import pandas as pd
import numpy as mp
#import dateutil
import random
from matplotlib import pyplot
pd.set_option('colheader_justify', 'left')   # FOR TABLE <th>
#pd.set_option('colheader_justify', 'left')

In [293]:
def color_format(value):
    """
    Colors scheme can be changed
    """

    if value == "Active" or value == "High":
        color = 'red'
    elif value == "Closed":
        color = 'green'
    elif value == "In Test" or value == 'Resolved':
        color = 'orange'
    elif value == 'Medium':
        color = '#bf00ff'
    elif value == 'Low':
        color = '#00c0ff'
    elif value == 'Deferred':
        color = 'blue'
    else:
        color = 'black'

          
    return 'font-size:11pt; font-family: Arial;border-collapse: collapse;border: 1px solid silver;text-align: left; color: %s' % color

In [334]:
def Generate_Status_Report():
    
    includeClosedBugs = True
    
    df_bugs = pd.read_csv('February Release - Combined3.csv', 
                          usecols=['Iteration Path','State','Severity'])[['Iteration Path','State','Severity']]
    df_bugs["Iteration Path"] = df_bugs["Iteration Path"].str.split('\\').str[-1]
    df_bugs.Severity = df_bugs["Severity"].str.split('- ').str[-1]
    
    #Filtering the closed records
    if not includeClosedBugs:
        print("Note: Excluded the Closed bugs")
        df_bugs = df_bugs[(df_bugs.State != 'Closed')]
        
    
    #State_n_Severity_ExcludingClosed =df_bugs.groupby(["Iteration Path", "State","Severity"])["Severity"].count() #.size()
    State_n_Severity = df_bugs.groupby(["Iteration Path", "State","Severity"]).agg(Bug_Count=pd.NamedAgg(column="Severity", aggfunc="count"))
    
    return State_n_Severity

PrintReport = Generate_Status_Report()

In [351]:
tbl = PrintReport.reset_index()
tbl[['Iteration Path']] = tbl[["Iteration Path"]].where(tbl[["Iteration Path"]]. \
                                                        apply(lambda ip: ip != ip.shift()), '')
tbl.style.applymap(color_format)

,Iteration Path,State,Severity,Bug_Count
0,Application Discovery,Active,Low,1
1,Down Time Ticker,Active,Low,1
2,,Active,Medium,1
3,,Deferred,Low,1
4,Regrouping of Tiles,Active,Low,1
5,,Closed,Low,1
6,,Deferred,Low,2
7,,In Test,Low,1
8,,Resolved,Low,1
9,Self Support,Active,Low,2


<h1>Convert the Bug Report into HTML </h1>

In [325]:
pd.set_option('colheader_justify', 'left')   # FOR TABLE <th>

html_string = '''
<html>

  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
  <h1> Daily Status Report </h1>
    {table}
  </body>
</html>.
'''


# OUTPUT AN HTML FILE
with open('myhtml.html', 'w') as f:
    f.write(html_string.format(table=tbl.to_html(classes='mystyle')))